# PyCUDA basics.

%install_ext http://raw.github.com/jrjohansson/version_information/master/version_information.py

#to install, execute in a cell: 
%install_ext https://raw.github.com/minrk/ipython_extensions/master/nbtoc.py
%load_ext nbtoc
%nbtoc

## Versions

In [2]:
%load_ext version_information
%version_information numpy, scipy, matplotlib, sympy, pycuda, pycula

Software versions
Python 2.7.6 64bit [GCC 4.8.2]
IPython 5.1.0
OS Linux 3.19.0 59 generic x86_64 with Ubuntu 14.04 trusty
numpy 1.11.1
scipy 0.17.1
matplotlib 2.0.0b3+2000.g22aa800
sympy 1.0
pycuda 2016.1
pycula The 'pycula' distribution was not found and is required by the application
Wed Sep 21 11:23:34 2016 CDT

In [3]:
import pycuda
import numpy as np

In [4]:
# Comandos de bash
!ls

0_Simple			 __pycache__
2013-12-03-Crank_Nicolson.ipynb  pycuda-helpers.hpp
addCPU				 README.md
Animation.ipynb			 README.md~
BuildTesting.ipynb		 SchrodingerBOX.pdf
cpua				 Sesion1_Introduccion_Python.ipynb
cpuAdd.c			 Sesion1_Test.ipynb
cpuAdd.c~			 Sesion2_Intento_de_solucion_GPE.ipynb
CUDAkernelsCONS.cu		 Sesión3.1_ExploreGPU.ipynb
CUDAkernelsCONS.cu~		 Sesion3_CUDA&PyCuda1st.ipynb
CUDAkernels.cu			 Sesion4_PyCUDA_2nd.ipynb
CUDAkernels.cu~			 Sesion5.5_MemoriasCUDA.ipynb
CUDATools.py			 Sesion5_MemoriasCUDA.ipynb
CUDATools.pyc			 Sesion6_PDE_CUDA_670.ipynb
ExploringLayeredMemory.ipynb	 Sesion6_PDE_CUDA_780.ipynb
gpu				 Sesion6_PDE_CUDA.ipynb
gpuAdd				 Sesion6_PDE_CUDA-Tesla.ipynb
gpuAdd.cu			 Sesion7_Librerias(FFT) GTX 780.ipynb
gpuAdd.cu~			 Sesion7_Librerias(FFT)-Tesla C2075.ipynb
imagen				 Sesion8_PDE_CUDA_Schrodinger.ipynb
intervalo.py			 Some_bench.ipynb
intervalo.pyc			 taylor.py
libpeerconnection.log		 taylor.py~
License.txt			 taylor.pyc
Longuet-Higgins.ipynb		 testing

In [5]:
!git status

En la rama master
Su rama está delante de «origin/master» para 1 commit.
  (use "git push" to publish your local commits)
Cambios no preparados para el commit:
  (use «git add/rm <archivo>...» para actualizar lo que se confirmará)
  (use «git checkout -- <archivo>...» para descartar cambios en el directorio de trabajo)

	modified:   Sesion1_Introduccion_Python.ipynb
	modified:   Sesion5_MemoriasCUDA.ipynb
	modified:   Sesion6_PDE_CUDA.ipynb
	modified:   Sesion6_PDE_CUDA_780.ipynb
	deleted:    Sesion7_LibreriasGPU.ipynb
	modified:   "Sesi\303\263n3.1_ExploreGPU.ipynb"

Archivos sin seguimiento:
  (use «git add <archivo>...» para incluir en lo que se ha de confirmar)

	2013-12-03-Crank_Nicolson.ipynb
	BuildTesting.ipynb
	CUDATools.py
	CUDATools.pyc
	CUDAkernelsCONS.cu
	CUDAkernelsCONS.cu~
	ExploringLayeredMemory.ipynb
	MYpycuda-helpers.hpp
	MeshingXYZ.ipynb
	Sesion5.5_MemoriasCUDA.ipynb
	Sesion6_PDE_CUDA-Tesla.ipynb
	Sesion6_PDE_CUDA_670.ipynb
	Sesion7_Librerias(FFT) GTX 780.ipynb
	Sesio

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2015 NVIDIA Corporation
Built on Tue_Aug_11_14:27:32_CDT_2015
Cuda compilation tools, release 7.5, V7.5.17


## Exploring your GPU device.

Listing devices:

In [7]:
from pycuda import autoinit
from pycuda.tools import DeviceData

In [8]:
specs = DeviceData()

In [9]:
print 'Max threads per block = ',specs.max_threads

Max threads per block =  1024


In [10]:
print 'Warp size            =', specs.warp_size
print 'Warps per MP         =', specs.warps_per_mp
print 'Thread Blocks per MP =', specs.thread_blocks_per_mp
print 'Registers            =', specs.registers
print 'Shared memory        =', specs.shared_memory
print 'Granularity ??       =', specs.smem_granularity

Warp size            = 32
Warps per MP         = 48
Thread Blocks per MP = 8
Registers            = 65536
Shared memory        = 49152
Granularity ??       = 32


    GTX 670
    Warp size            = 32
    Warps per MP         = 48
    Thread Blocks per MP = 8
    Registers            = 65536
    Shared memory        = 49152
    Granularity ??       = 32

    GTX 860m
    Warp size            = 32
    Warps per MP         = 48
    Thread Blocks per MP = 8
    Registers            = 65536
    Shared memory        = 49152
    Granularity ??       = 32
    

Other way

In [11]:
import pycuda.driver as drv

In [12]:
drv.init()

In [13]:
drv.get_version()

(7, 5, 0)

In [14]:
devn = drv.Device.count()
print 'Localized GPUs =',devn

Localized GPUs = 1


In [15]:
devices = []
for i in range(devn):
    devices.append(drv.Device(i))

All you want to know about your GPU, but you're afraid to ask!

In [16]:
for sp in devices:
    print 'Name = ',sp.name()
    print 'PCI Bus = ',sp.pci_bus_id()
    print 'Compute Capability = ',sp.compute_capability()
    print 'Total Memory = ',sp.total_memory()/(2.**20) , 'MBytes'
    attr = sp.get_attributes()
    for j in range(len(attr.items())):
        print attr.items()[j]#,'Bytes (when apply)'
    print '------------------'
    print '------------------'

Name =  GeForce GTX 860M
PCI Bus =  0000:01:00.0
Compute Capability =  (5, 0)
Total Memory =  2047.875 MBytes
(pycuda._driver.device_attribute.MAX_THREADS_PER_BLOCK, 1024)
(pycuda._driver.device_attribute.MAX_BLOCK_DIM_X, 1024)
(pycuda._driver.device_attribute.MAX_BLOCK_DIM_Y, 1024)
(pycuda._driver.device_attribute.MAX_BLOCK_DIM_Z, 64)
(pycuda._driver.device_attribute.MAX_GRID_DIM_X, 2147483647)
(pycuda._driver.device_attribute.MAX_GRID_DIM_Y, 65535)
(pycuda._driver.device_attribute.MAX_GRID_DIM_Z, 65535)
(pycuda._driver.device_attribute.MAX_SHARED_MEMORY_PER_BLOCK, 49152)
(pycuda._driver.device_attribute.TOTAL_CONSTANT_MEMORY, 65536)
(pycuda._driver.device_attribute.WARP_SIZE, 32)
(pycuda._driver.device_attribute.MAX_PITCH, 2147483647)
(pycuda._driver.device_attribute.MAX_REGISTERS_PER_BLOCK, 65536)
(pycuda._driver.device_attribute.CLOCK_RATE, 1019500)
(pycuda._driver.device_attribute.TEXTURE_ALIGNMENT, 512)
(pycuda._driver.device_attribute.GPU_OVERLAP, 1)
(pycuda._driver.device_attri

MAX_THREADS_PER_BLOCK, 1024

For example for a 3D mesh (less optimal), we only have avaiable $$8\times 8\times 8 = 512 \,simetric$$ 
 $$8\times 8\times 16 = 1024 \,cilindrical$$
block size per dimension = 8 or 16.
In 2D case the optimal value is:
$$32\times32 = 1024$$
In last case $$1024$$


MAX_THREADS_PER_MULTIPROCESSOR, $1536 = 3*2^9$

If we can take this literally, we can process in one processor about 3 meshes of $8\times8\times8$, or three blocks of 3D meshes. With this result, we can evaluate the eficience comparing cilindrical and simetric performance


### Now your device has ..

In [17]:
drv.mem_get_info()[0]/(2.**20),'MB of Free Memory',drv.mem_get_info()[1]/(2.**20),'MB Total Memory'

(2006.47265625, 'MB of Free Memory', 2047.875, 'MB Total Memory')

Let's think in array sizes. For example a float of 4 bytes length:

In [18]:
print 'Linear max length:', drv.mem_get_info()[0]/(4)
print '2D max length    :', np.sqrt(drv.mem_get_info()[0]/(4))
print '3D max length    :', np.power(drv.mem_get_info()[0]/(4),1./3.)

Linear max length: 525984768
2D max length    : 22934.3578066
3D max length    : 807.218405712


In [19]:
!nvidia-smi

Wed Sep 21 11:28:21 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 361.45.18              Driver Version: 361.45.18                 |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 860M    Off  | 0000:01:00.0     Off |                  N/A |
| N/A   43C    P8    N/A /  N/A |     41MiB /  2047MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  